<a href="https://colab.research.google.com/github/felipe-parodi/imitation-learning-cis522/blob/main/Ridge_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Add link to drive if using Colab. Run locally for visualizing agent. 

In [ ]:
# SESSION PARAMETERS
use_colab    = False 
train_model  = 1    # 0: evaluate only
new_training = 0    # 0: load model and continue training; 1: Start from scratch

if use_colab:
  from google.colab import drive
  drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# @markdown ### Install pybullet (Colab only)
if use_colab:
  !pip install pybullet
  !apt-get update
  !apt -q install imagemagick
  !apt install -q xvfb
  !pip install -q pyvirtualdisplay

In [ ]:
# @markdown ### Install dependencies
import time
import os
import glob
os.environ['MESA_GL_VERSION_OVERRIDE'] = '3.3'
os.environ['MESA_GLSL_VERSION_OVERRIDE'] = '330'
import json
import numpy as np
import pandas as pd
import random
import sys
import time
from builtins import super
from collections import namedtuple
from tqdm.auto import tqdm
from copy import deepcopy
from typing import NamedTuple
import pdb
import scipy.signal

# Pybullet
import pybullet as p
if use_colab:
  p.connect(p.DIRECT)
import pybullet_data
from   pybullet_utils.arg_parser import ArgParser
from   pybullet_envs.deep_mimic.env.pybullet_deep_mimic_env import PyBulletDeepMimicEnv
from pybullet_envs.deep_mimic.learning.path import *
  

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, lasso_path, enet_path
from sklearn.metrics import mean_squared_error, r2_score

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from   torch.autograd import Variable as V
from   torch.distributions import Normal
from   torch.optim import Adam

# Plotting 
import matplotlib.pyplot as plt
from matplotlib import pylab
from matplotlib import animation

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



     |████████████████████████████████| 89.3MB 56kB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

### Build environment

In [ ]:
# Define some parameters
timestep     = 1. / 240.
animating    = True
step         = False
total_reward = 0
steps        = 0

# Build arg parser
arg_parser   = ArgParser()
arg_file     = "run_humanoid3d_walk_args.txt"
argpath      = pybullet_data.getDataPath() + "/args/" + arg_file
succ         = arg_parser.load_file(argpath)
project_path = "/content/gdrive/MyDrive/Colab Notebooks/Penn Deep Learning Course/RL Project"
expert_data  = os.path.join(project_path,"state_action_deepmimic_850reward.npz")

# Buiild env
env = PyBulletDeepMimicEnv(arg_parser, enable_draw=False)

Initialization strategy: InitializationStrategy.RANDOM
motion_file= data/motions/humanoid3d_walk.txt
LOADING humanoid!


### Load expert data and run ridge regression

In [ ]:
# Load expert data
with np.load(expert_data,allow_pickle=True) as data:
    action = data['actions']
    state  = data['states']
# Get dimensions
act_dim   = action.shape
state_dim = state.shape

# Split into testing and training
X_train, X_test, y_train, y_test = train_test_split(state,action, test_size=0.15, random_state=123)

# Fit ridge regression
rr = Ridge(alpha=1) 
rr.fit(X_train,y_train)
y_rr = rr.predict(X_test)
mse_rr = mean_squared_error(y_rr,y_test)
r2_rr = r2_score(y_rr,y_test)

print(mse_rr, r2_rr)

# Fit lasso just in case
ll = Lasso(alpha=1) 
ll.fit(X_train,y_train)
y_ll = ll.predict(X_test)
mse_ll = mean_squared_error(y_ll,y_test)
r2_ll = r2_score(y_ll,y_test)

### Evaluate model using N random starts

In [ ]:
# Simulate agent in environment
def get_state_from_action(env,action,timestep):
     # Convert action to pose: calls self.desiredPose = self._humanoid.convertActionToPose(action)
     env.set_action(0, action)
     # Update env using PD controller to get joint torques: self._humanoid.computeAndApplyPDForces(self.desiredPose, maxForces=maxForces)
     env.update(timestep)
     # Get current state
     state = env.record_state(0)
     # Get simple reward for being alive else none
     if env.is_episode_end():
            reward = env.calc_reward(0) #0
            done   = True
     else: 
            reward = env.calc_reward(0) #1
            done   = False
     return state, reward, done

num_iter = 1000
all_reward = []
num_steps = []
for ii in range(num_iter):
    print(ii)
    ep_steps=300
    total_reward = 0
    state = env.reset()
    state = env.record_state(0)
    for s in range(ep_steps):
        action = rr.predict(state.reshape(1, -1))
        next_state, reward, done = get_state_from_action(env,action[0],timestep)
        total_reward += reward
        state = next_state
        if done:
            num_steps.append(s)
            all_reward.append(total_reward)
            break
            
    
np.savez("ridgeregression_1000iterations_DeepMimicHumanoid",all_reward=all_reward,num_steps=num_steps)